In [11]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
import movingpandas as mpd
from geographiclib.geodesic import Geodesic
import warnings
warnings.filterwarnings('ignore')

import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta

mpd.show_versions()


MovingPandas 0.9.rc2

SYSTEM INFO
-----------
python     : 3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:40:17)  [GCC 9.4.0]
executable : /home/dy/anaconda3/envs/mp/bin/python
machine    : Linux-5.8.0-44-generic-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : 3.10.1
GEOS lib   : /home/dy/anaconda3/envs/mp/lib/libgeos_c.so
GDAL       : 3.4.1
GDAL data dir: /home/dy/anaconda3/envs/mp/share/gdal
PROJ       : 8.2.1
PROJ data dir: /home/dy/anaconda3/envs/mp/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.10.2
pandas     : 1.3.5
fiona      : 1.8.20
numpy      : 1.21.5
shapely    : 1.8.0
rtree      : 0.9.7
pyproj     : 3.3.0
matplotlib : 3.5.1
mapclassify: 2.4.3
geopy      : 2.2.0
holoviews  : 1.14.7
hvplot     : 0.7.3
geoviews   : 1.9.3


In [12]:
FSIZE = 500

In [13]:
# filepath
fp = pd.read_csv('data/dtg_20191224.csv')
fp.head

<bound method NDFrame.head of               timestamp   vehicleid   latitude   longitude   speed
0        20191224000915         228  36.357918  127.370253      80
1        20191224000916         228  36.357918  127.370001      79
2        20191224000917         228  36.357920  127.369756      78
3        20191224000918         228  36.357920  127.369513      77
4        20191224000919         228  36.357916  127.369275      75
...                 ...         ...        ...         ...     ...
7932695  20191224091611         457  36.334336  127.370043      29
7932696  20191224091612         457  36.334321  127.370130      28
7932697  20191224091613         457  36.334305  127.370215      28
7932698  20191224091614         457  36.334286  127.370298      27
7932699  20191224091615         457  36.334266  127.370380      25

[7932700 rows x 5 columns]>

In [14]:
fp.timestamp = fp.timestamp.astype(str)

In [15]:
fp['timestamp'] = pd.to_datetime(fp['timestamp'], format="%Y%m%d%H%M%S")

In [16]:
fp.columns = ['t', 'trajectory_id', 'y', 'x', 'speed']

In [17]:
fp.head()

,t,trajectory_id,y,x,speed
0,2019-12-24 00:09:15,228,36.357918,127.370253,80
1,2019-12-24 00:09:16,228,36.357918,127.370001,79
2,2019-12-24 00:09:17,228,36.357920,127.369756,78
3,2019-12-24 00:09:18,228,36.357920,127.369513,77
4,2019-12-24 00:09:19,228,36.357916,127.369275,75


In [18]:
fp0001 = fp[(fp['t']>'2019-12-24 00:00:00') & (fp['t']<'2019-12-24 01:00:00')]
fp0102 = fp[(fp['t']>'2019-12-24 01:00:00') & (fp['t']<'2019-12-24 02:00:00')]
fp0203 = fp[(fp['t']>'2019-12-24 02:00:00') & (fp['t']<'2019-12-24 03:00:00')]
fp0304 = fp[(fp['t']>'2019-12-24 03:00:00') & (fp['t']<'2019-12-24 04:00:00')]
fp0405 = fp[(fp['t']>'2019-12-24 04:00:00') & (fp['t']<'2019-12-24 05:00:00')]
fp0506 = fp[(fp['t']>'2019-12-24 05:00:00') & (fp['t']<'2019-12-24 06:00:00')]
fp0607 = fp[(fp['t']>'2019-12-24 06:00:00') & (fp['t']<'2019-12-24 07:00:00')]
fp0708 = fp[(fp['t']>'2019-12-24 07:00:00') & (fp['t']<'2019-12-24 08:00:00')]
fp0809 = fp[(fp['t']>'2019-12-24 08:00:00') & (fp['t']<'2019-12-24 09:00:00')]
fp0910 = fp[(fp['t']>'2019-12-24 09:00:00') & (fp['t']<'2019-12-24 10:00:00')]
fp1011 = fp[(fp['t']>'2019-12-24 10:00:00') & (fp['t']<'2019-12-24 11:00:00')]
fp1112 = fp[(fp['t']>'2019-12-24 11:00:00') & (fp['t']<'2019-12-24 12:00:00')]
fp1213 = fp[(fp['t']>'2019-12-24 12:00:00') & (fp['t']<'2019-12-24 13:00:00')]
fp1314 = fp[(fp['t']>'2019-12-24 13:00:00') & (fp['t']<'2019-12-24 14:00:00')]
fp1415 = fp[(fp['t']>'2019-12-24 14:00:00') & (fp['t']<'2019-12-24 15:00:00')]
fp1516 = fp[(fp['t']>'2019-12-24 15:00:00') & (fp['t']<'2019-12-24 16:00:00')]
fp1617 = fp[(fp['t']>'2019-12-24 16:00:00') & (fp['t']<'2019-12-24 17:00:00')]
fp1718 = fp[(fp['t']>'2019-12-24 17:00:00') & (fp['t']<'2019-12-24 18:00:00')]
fp1819 = fp[(fp['t']>'2019-12-24 18:00:00') & (fp['t']<'2019-12-24 19:00:00')]
fp1920 = fp[(fp['t']>'2019-12-24 19:00:00') & (fp['t']<'2019-12-24 20:00:00')]
fp2021 = fp[(fp['t']>'2019-12-24 20:00:00') & (fp['t']<'2019-12-24 21:00:00')]
fp2122 = fp[(fp['t']>'2019-12-24 21:00:00') & (fp['t']<'2019-12-24 22:00:00')]
fp2223 = fp[(fp['t']>'2019-12-24 22:00:00') & (fp['t']<'2019-12-24 23:00:00')]
fp2324 = fp[(fp['t']>'2019-12-24 23:00:00') & (fp['t']<'2019-12-24 23:59:59')]

fp_all = fp[(fp['t']>'2019-12-24 00:00:00') & (fp['t']<'2019-12-24 23:59:59')]

In [19]:
traj_collection_0001 = mpd.TrajectoryCollection(fp0001, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0102 = mpd.TrajectoryCollection(fp0102, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0203 = mpd.TrajectoryCollection(fp0203, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0304 = mpd.TrajectoryCollection(fp0304, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0405 = mpd.TrajectoryCollection(fp0405, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0506 = mpd.TrajectoryCollection(fp0506, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0607 = mpd.TrajectoryCollection(fp0607, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0708 = mpd.TrajectoryCollection(fp0708, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0809 = mpd.TrajectoryCollection(fp0809, 'trajectory_id',t='t', x='x', y='y')
traj_collection_0910 = mpd.TrajectoryCollection(fp0910, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1011 = mpd.TrajectoryCollection(fp1011, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1112 = mpd.TrajectoryCollection(fp1112, 'trajectory_id',t='t', x='x', y='y')

traj_collection_1213 = mpd.TrajectoryCollection(fp1213, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1314 = mpd.TrajectoryCollection(fp1314, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1415 = mpd.TrajectoryCollection(fp1415, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1516 = mpd.TrajectoryCollection(fp1516, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1617 = mpd.TrajectoryCollection(fp1617, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1718 = mpd.TrajectoryCollection(fp1718, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1819 = mpd.TrajectoryCollection(fp1819, 'trajectory_id',t='t', x='x', y='y')
traj_collection_1920 = mpd.TrajectoryCollection(fp1920, 'trajectory_id',t='t', x='x', y='y')
traj_collection_2021 = mpd.TrajectoryCollection(fp2021, 'trajectory_id',t='t', x='x', y='y')
traj_collection_2122 = mpd.TrajectoryCollection(fp2122, 'trajectory_id',t='t', x='x', y='y')
traj_collection_2223 = mpd.TrajectoryCollection(fp2223, 'trajectory_id',t='t', x='x', y='y')
traj_collection_2324 = mpd.TrajectoryCollection(fp2324, 'trajectory_id',t='t', x='x', y='y')
traj_collection_all = mpd.TrajectoryCollection(fp_all, 'trajectory_id',t='t', x='x', y='y')

In [ ]:
print(traj_collection_all)

## [DTG] 대전시 차량 흐름도 

주행경로 시각화를 확인한 결과, 통행량이 많은 포인트와 구간에 가중치를 부여해 트래픽이 몰리는 지점과 구간을 시각화 하고 싶음.

In [ ]:
traj_collection_all.plot(column='trajectory_id', legend=True, figsize=(18,14), alpha=0.7)

#### 거리를 기준으로 일반화 

In [20]:
generalized_0001 = mpd.MinDistanceGeneralizer(traj_collection_0001).generalize(tolerance=100)
generalized_0102 = mpd.MinDistanceGeneralizer(traj_collection_0102).generalize(tolerance=100)
generalized_0203 = mpd.MinDistanceGeneralizer(traj_collection_0203).generalize(tolerance=100)
generalized_0304 = mpd.MinDistanceGeneralizer(traj_collection_0304).generalize(tolerance=100)
generalized_0405 = mpd.MinDistanceGeneralizer(traj_collection_0405).generalize(tolerance=100)
generalized_0506 = mpd.MinDistanceGeneralizer(traj_collection_0506).generalize(tolerance=100)
generalized_0607 = mpd.MinDistanceGeneralizer(traj_collection_0607).generalize(tolerance=100)
generalized_0708 = mpd.MinDistanceGeneralizer(traj_collection_0708).generalize(tolerance=100)
generalized_0809 = mpd.MinDistanceGeneralizer(traj_collection_0809).generalize(tolerance=100)
generalized_0910 = mpd.MinDistanceGeneralizer(traj_collection_0910).generalize(tolerance=100)
generalized_1011 = mpd.MinDistanceGeneralizer(traj_collection_1011).generalize(tolerance=100)
generalized_1112 = mpd.MinDistanceGeneralizer(traj_collection_1112).generalize(tolerance=100)

generalized_1213 = mpd.MinDistanceGeneralizer(traj_collection_1213).generalize(tolerance=100)
generalized_1314 = mpd.MinDistanceGeneralizer(traj_collection_1314).generalize(tolerance=100)
generalized_1415 = mpd.MinDistanceGeneralizer(traj_collection_1415).generalize(tolerance=100)
generalized_1516 = mpd.MinDistanceGeneralizer(traj_collection_1516).generalize(tolerance=100)
generalized_1617 = mpd.MinDistanceGeneralizer(traj_collection_1617).generalize(tolerance=100)
generalized_1718 = mpd.MinDistanceGeneralizer(traj_collection_1718).generalize(tolerance=100)
generalized_1819 = mpd.MinDistanceGeneralizer(traj_collection_1819).generalize(tolerance=100)
generalized_1920 = mpd.MinDistanceGeneralizer(traj_collection_1920).generalize(tolerance=100)
generalized_2021 = mpd.MinDistanceGeneralizer(traj_collection_2021).generalize(tolerance=100)
generalized_2122 = mpd.MinDistanceGeneralizer(traj_collection_2122).generalize(tolerance=100)
generalized_2223 = mpd.MinDistanceGeneralizer(traj_collection_2223).generalize(tolerance=100)
generalized_2324 = mpd.MinDistanceGeneralizer(traj_collection_2324).generalize(tolerance=100)

In [ ]:
print(generalized_0910)
print(generalized_1819)

In [ ]:
# 시간을 기준으로 일반화 
# generalized_0001 = mpd.MinTimeDeltaGeneralizer(traj_collection_0001).generalize(tolerance=timedelta(minutes=10))
# generalized_0001

#### 

In [21]:
aggregator_0001 = mpd.TrajectoryCollectionAggregator(generalized_0001, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0102 = mpd.TrajectoryCollectionAggregator(generalized_0102, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0203 = mpd.TrajectoryCollectionAggregator(generalized_0203, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0304 = mpd.TrajectoryCollectionAggregator(generalized_0304, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0405 = mpd.TrajectoryCollectionAggregator(generalized_0405, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0506 = mpd.TrajectoryCollectionAggregator(generalized_0506, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0607 = mpd.TrajectoryCollectionAggregator(generalized_0607, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0708 = mpd.TrajectoryCollectionAggregator(generalized_0708, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0809 = mpd.TrajectoryCollectionAggregator(generalized_0809, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_0910 = mpd.TrajectoryCollectionAggregator(generalized_0910, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1011 = mpd.TrajectoryCollectionAggregator(generalized_1011, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1112 = mpd.TrajectoryCollectionAggregator(generalized_1112, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))

aggregator_1213 = mpd.TrajectoryCollectionAggregator(generalized_1213, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1314 = mpd.TrajectoryCollectionAggregator(generalized_1314, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1415 = mpd.TrajectoryCollectionAggregator(generalized_1415, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1516 = mpd.TrajectoryCollectionAggregator(generalized_1516, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1617 = mpd.TrajectoryCollectionAggregator(generalized_1617, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1718 = mpd.TrajectoryCollectionAggregator(generalized_1718, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1819 = mpd.TrajectoryCollectionAggregator(generalized_1819, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_1920 = mpd.TrajectoryCollectionAggregator(generalized_1920, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_2021 = mpd.TrajectoryCollectionAggregator(generalized_2021, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_2122 = mpd.TrajectoryCollectionAggregator(generalized_2122, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_2223 = mpd.TrajectoryCollectionAggregator(generalized_2223, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))
aggregator_2324 = mpd.TrajectoryCollectionAggregator(generalized_2324, min_distance=10, max_distance=500, min_stop_duration=timedelta(minutes=10))

In [ ]:
print(aggregator_0910)
print(aggregator_1819)

In [22]:
pts_0001 = aggregator_0001.get_significant_points_gdf()
pts_0102 = aggregator_0102.get_significant_points_gdf()
pts_0203 = aggregator_0203.get_significant_points_gdf()
pts_0304 = aggregator_0304.get_significant_points_gdf()
pts_0405 = aggregator_0405.get_significant_points_gdf()
pts_0506 = aggregator_0506.get_significant_points_gdf()
pts_0607 = aggregator_0607.get_significant_points_gdf()
pts_0708 = aggregator_0708.get_significant_points_gdf()
pts_0809 = aggregator_0809.get_significant_points_gdf()
pts_0910 = aggregator_0910.get_significant_points_gdf()
pts_1011 = aggregator_1011.get_significant_points_gdf()
pts_1112 = aggregator_1112.get_significant_points_gdf()

pts_1213 = aggregator_1213.get_significant_points_gdf()
pts_1314 = aggregator_1314.get_significant_points_gdf()
pts_1415 = aggregator_1415.get_significant_points_gdf()
pts_1516 = aggregator_1516.get_significant_points_gdf()
pts_1617 = aggregator_1617.get_significant_points_gdf()
pts_1718 = aggregator_1718.get_significant_points_gdf()
pts_1819 = aggregator_1819.get_significant_points_gdf()
pts_1920 = aggregator_1920.get_significant_points_gdf()
pts_2021 = aggregator_2021.get_significant_points_gdf()
pts_2122 = aggregator_2122.get_significant_points_gdf()
pts_2223 = aggregator_2223.get_significant_points_gdf()
pts_2324 = aggregator_2324.get_significant_points_gdf()

In [ ]:
print(pts_0910)
print(pts_1819)

### cluster

다중 교차하는 지점에 대한 clustering

In [23]:
clusters_0001 = aggregator_0001.get_clusters_gdf()
clusters_0102 = aggregator_0102.get_clusters_gdf()
clusters_0203 = aggregator_0203.get_clusters_gdf()
clusters_0304 = aggregator_0304.get_clusters_gdf()
clusters_0405 = aggregator_0405.get_clusters_gdf()
clusters_0506 = aggregator_0506.get_clusters_gdf()
clusters_0607 = aggregator_0607.get_clusters_gdf()
clusters_0708 = aggregator_0708.get_clusters_gdf()
clusters_0809 = aggregator_0809.get_clusters_gdf()
clusters_0910 = aggregator_0910.get_clusters_gdf()
clusters_1011 = aggregator_1011.get_clusters_gdf()
clusters_1112 = aggregator_1112.get_clusters_gdf()

clusters_1213 = aggregator_1213.get_clusters_gdf()
clusters_1314 = aggregator_1314.get_clusters_gdf()
clusters_1415 = aggregator_1415.get_clusters_gdf()
clusters_1516 = aggregator_1516.get_clusters_gdf()
clusters_1617 = aggregator_1617.get_clusters_gdf()
clusters_1718 = aggregator_1718.get_clusters_gdf()
clusters_1819 = aggregator_1819.get_clusters_gdf()
clusters_1920 = aggregator_1920.get_clusters_gdf()
clusters_2021 = aggregator_2021.get_clusters_gdf()
clusters_2122 = aggregator_2122.get_clusters_gdf()
clusters_2223 = aggregator_2223.get_clusters_gdf()
clusters_2324 = aggregator_2324.get_clusters_gdf()

print(clusters_0910)
print(clusters_1819)

                      geometry    n
0   POINT (127.36954 36.35269)  321
1   POINT (127.36869 36.33451)   92
2   POINT (127.35709 36.33253)   94
3   POINT (127.34927 36.33243)  149
4   POINT (127.34547 36.33218)   48
..                         ...  ...
60  POINT (127.33834 36.32934)   29
61  POINT (127.35850 36.36453)   75
62  POINT (127.36822 36.36257)   70
63  POINT (127.33291 36.32876)    5
64  POINT (127.36465 36.34505)   10

[65 rows x 2 columns]
                      geometry    n
0   POINT (127.32793 36.36741)    0
1   POINT (127.36925 36.35801)  203
2   POINT (127.33384 36.36587)   71
3   POINT (127.33758 36.36330)  178
4   POINT (127.34071 36.36164)   76
..                         ...  ...
58  POINT (127.31807 36.35492)    1
59  POINT (127.32475 36.35280)    3
60  POINT (127.32935 36.35943)   28
61  POINT (127.32531 36.35955)   60
62  POINT (127.36790 36.33023)    1

[63 rows x 2 columns]


In [24]:
clusters = pd.concat([clusters_0001, clusters_0102, clusters_0203, clusters_0304, clusters_0405, clusters_0506, clusters_0607,
                     clusters_0708, clusters_0809, clusters_0910, clusters_1011, clusters_1112, clusters_1213, clusters_1314, 
                     clusters_1415, clusters_1516, clusters_1617, clusters_1718, clusters_1819, clusters_1920, clusters_2021, 
                     clusters_2122, clusters_2223, clusters_2324])

In [25]:
clusters

,geometry,n
0,POINT (127.34352 36.35854),776
1,POINT (127.34794 36.32776),182
2,POINT (127.33806 36.36370),1328
3,POINT (127.33329 36.36604),687
4,POINT (127.33936 36.36781),149
...,...,...
52,POINT (127.35756 36.36212),278
53,POINT (127.36347 36.33435),65
54,POINT (127.34380 36.33940),69
55,POINT (127.35787 36.33274),192


In [26]:
clusters.to_csv('clusters_191224.csv')

### flow

- cluster 구간에 대한 flow(origin -> dest) 생성
-  date type merge 

In [27]:
# before -> after flows
flows_0001 = aggregator_0001.get_flows_gdf()
flows_0102 = aggregator_0102.get_flows_gdf()
flows_0203 = aggregator_0203.get_flows_gdf()
flows_0304 = aggregator_0304.get_flows_gdf()
flows_0405 = aggregator_0405.get_flows_gdf()
flows_0506 = aggregator_0506.get_flows_gdf()
flows_0607 = aggregator_0607.get_flows_gdf()
flows_0708 = aggregator_0708.get_flows_gdf()
flows_0809 = aggregator_0809.get_flows_gdf()
flows_0910 = aggregator_0910.get_flows_gdf()
flows_1011 = aggregator_1011.get_flows_gdf()
flows_1112 = aggregator_1112.get_flows_gdf()

flows_1213 = aggregator_1213.get_flows_gdf()
flows_1314 = aggregator_1314.get_flows_gdf()
flows_1415 = aggregator_1415.get_flows_gdf()
flows_1516 = aggregator_1516.get_flows_gdf()
flows_1617 = aggregator_1617.get_flows_gdf()
flows_1718 = aggregator_1718.get_flows_gdf()
flows_1819 = aggregator_1819.get_flows_gdf()
flows_1920 = aggregator_1920.get_flows_gdf()
flows_2021 = aggregator_2021.get_flows_gdf()
flows_2122 = aggregator_2122.get_flows_gdf()
flows_2223 = aggregator_2223.get_flows_gdf()
flows_2324 = aggregator_2324.get_flows_gdf()

In [28]:
flows_0001['time'] = '2019-12-24 01:00:00'
flows_0102['time'] = '2019-12-24 02:00:00'
flows_0203['time'] = '2019-12-24 03:00:00'
flows_0304['time'] = '2019-12-24 04:00:00'
flows_0405['time'] = '2019-12-24 05:00:00'
flows_0506['time'] = '2019-12-24 06:00:00'
flows_0607['time'] = '2019-12-24 07:00:00'
flows_0708['time'] = '2019-12-24 08:00:00'
flows_0809['time'] = '2019-12-24 09:00:00'
flows_0910['time'] = '2019-12-24 10:00:00'
flows_1011['time'] = '2019-12-24 11:00:00'
flows_1112['time'] = '2019-12-24 12:00:00'
flows_1213['time'] = '2019-12-24 13:00:00'

flows_1314['time'] = '2019-12-24 14:00:00'
flows_1415['time'] = '2019-12-24 15:00:00'
flows_1516['time'] = '2019-12-24 16:00:00'
flows_1617['time'] = '2019-12-24 17:00:00'
flows_1718['time'] = '2019-12-24 18:00:00'
flows_1819['time'] = '2019-12-24 19:00:00'
flows_1920['time'] = '2019-12-24 20:00:00'
flows_2021['time'] = '2019-12-24 21:00:00'
flows_2122['time'] = '2019-12-24 22:00:00'
flows_2223['time'] = '2019-12-24 23:00:00'
flows_2324['time'] = '2019-12-24 23:59:59'

In [29]:
flows_0809

,geometry,weight,time
0,"LINESTRING (127.36931 36.35856, 127.36414 36.3...",19,2019-12-24 09:00:00
1,"LINESTRING (127.36414 36.35585, 127.35859 36.3...",31,2019-12-24 09:00:00
2,"LINESTRING (127.35859 36.36023, 127.35248 36.3...",47,2019-12-24 09:00:00
3,"LINESTRING (127.35248 36.35925, 127.34532 36.3...",48,2019-12-24 09:00:00
4,"LINESTRING (127.34532 36.36143, 127.34556 36.3...",12,2019-12-24 09:00:00
...,...,...,...
230,"LINESTRING (127.36832 36.36300, 127.36307 36.3...",1,2019-12-24 09:00:00
231,"LINESTRING (127.33168 36.35782, 127.33750 36.3...",1,2019-12-24 09:00:00
232,"LINESTRING (127.36769 36.33527, 127.34726 36.3...",1,2019-12-24 09:00:00
233,"LINESTRING (127.34726 36.33218, 127.35802 36.3...",1,2019-12-24 09:00:00


In [30]:
flows = pd.concat([flows_0001, flows_0102, flows_0203, flows_0304, flows_0405, flows_0506, flows_0607,
                     flows_0708, flows_0809, flows_0910, flows_1011, flows_1112, flows_1213, flows_1314, 
                     flows_1415, flows_1516, flows_1617, flows_1718,flows_1819, flows_1920, flows_2021, 
                     flows_2122, flows_2223,flows_2324])

In [31]:
flows.head()

,geometry,weight,time
0,"LINESTRING (127.34352 36.35854, 127.33806 36.3...",27,2019-12-24 01:00:00
1,"LINESTRING (127.33806 36.36370, 127.33329 36.3...",188,2019-12-24 01:00:00
2,"LINESTRING (127.33329 36.36604, 127.33936 36.3...",12,2019-12-24 01:00:00
3,"LINESTRING (127.33936 36.36781, 127.33806 36.3...",79,2019-12-24 01:00:00
4,"LINESTRING (127.33806 36.36370, 127.34434 36.3...",170,2019-12-24 01:00:00


In [32]:
flows.to_csv('flows_191224.csv')

시간대별로 DTG 차량의 주행 경로를 확인하면 전체적인 흐름을 파악할 수는 있지만,    
**주행 경로가 겹치게 되는 구간이 존재할 때 flow가 중첩되어 차량의 이동경로(방향)를 자세히 확인할 수 없는 단점이 있음.** 
  
flows 데이터 프레임에 있는 geometry(before(lat, long) > after (lat, long) 컬럼으로 시간대별 차량이동 흐름 시각화를 진행해 해당 시간에 트래픽이 몰리는 지점의 포인트와 방향을 파악할 수 있도록 하는 것이 목적